<a href="https://colab.research.google.com/github/Iispar/hlt-project/blob/main/course_project_2023_template.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction to HLT 2023 Project (Template)

- Student(s) Name(s): Iiro Partanen
- Date: -
- Chosen Corpus: emotion
- Contributions (if group project):

### Corpus information

- Description of the chosen corpus: 
Emotion is a dataset of English Twitter messages with six basic emotions: anger, fear, joy, love, sadness, and surprise. For more detailed information please refer to the paper.
- Paper(s) and other published materials related to the corpus: 
  - CARER: Contextualized Affect Representations for Emotion Recognition (Saravia et al., EMNLP 2018)
  - https://paperswithcode.com/dataset/emotion
- State-of-the-art performance (best published results) on this corpus: 95% f1

---

## 1. Setup

In [8]:
!pip3 install -q transformers datasets evaluate
!pip install trankit
!pip install optuna
import datasets
import sklearn.feature_extraction
import torch
import transformers
import numpy as np
import evaluate
import optuna

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.7/365.7 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.3/212.3 kB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 7.5 MB/s eta 0:00:00


---

## 2. Data download and preprocessing

### 2.1. Download the corpus

In [9]:
dset = datasets.load_dataset("emotion");
# check it works
print(dset);

  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 16000
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
})


### 2.2. Preprocessing

In [10]:
# vectorizes one item
def vectorize_item(item):
    vectorized = vectorizer.transform([item["text"]]); # vectorize. Initialized below...
    non_zero_features = vectorized.nonzero()[1]; # get the nonzeros and we take only the columns of the nonzeros because our matrix is only one row.
    non_zero_features += 1; # index zero is for padding so let's avoid it by adding 1 to all.

    return {"input_ids":non_zero_features} 

In [11]:
dset.shuffle(); #shuffle dataset for safety

# vectorization.
vectorizer = sklearn.feature_extraction.text.CountVectorizer( # get the vectorizer.
    binary = True,
    max_features = 20000, # Selected 20k to start with.
    token_pattern = r"(?u)\b\w+\b" # Token pattern to include one char words.
    )

texts=[item["text"] for item in dset["train"]]; # get all texts from train
vectorizer.fit(texts); # fitting the vectorizer

# vectorize the whole dataset.
dset_tokenized = dset.map(vectorize_item,num_proc=4);
# check it works
print(dset_tokenized["train"][0]);

{'text': 'i didnt feel humiliated', 'label': 0, 'input_ids': [3620, 4931, 6438, 6495]}


In [12]:
# padding and batching
def collator(list_of_items):
    allLabels = [item["label"] for item in list_of_items]; # list of all labels.
    batch = {"labels": torch.tensor(allLabels)}; # create a tenstor for the item (batch)
    tensors = [];
    max_len = max(len(item["input_ids"]) for item in list_of_items); # longest example in the batch. Pad to here.
    for item in list_of_items:
        ids = torch.tensor(item["input_ids"]); # input ids to tensor
        padded = torch.nn.functional.pad(ids,(0,max_len-ids.shape[0])); # actual padding. Pads ids, from + to max with 0.
        tensors.append(padded); # appended ids to tensors
    batch["input_ids"] = torch.vstack(tensors); # stacks items as they are now same len. Now these are matrixes.
    return batch;

# check it works
batch=collator([dset_tokenized["train"][2],dset_tokenized["train"][7]])
print("Shape of labels:",batch["labels"].shape)
print("Shape of input_ids:",batch["input_ids"].shape)
print(batch["labels"])
print(batch["input_ids"])
     

Shape of labels: torch.Size([2])
Shape of input_ids: torch.Size([2, 13])
tensor([3, 4])
tensor([[    1,  4931,  5739,  5800,  6495,  6563,  8456, 10157, 13643, 15061,
             0,     0,     0],
        [    1,    34,   749,  2680,  4931,  6495,  7076,  7712,  8076,  9245,
          9325, 13339, 15116]])


---

## 3. Machine learning model

### 3.1. Model training

In [13]:
# Your code to train the machine learning model on the training set and evaluate the performance on the validation set here

# needs a config, we wil just pass it.
class MLPConfig(transformers.PretrainedConfig):
    pass;

# model
class MLP(transformers.PreTrainedModel):
      config_class = MLPConfig; # sets config
      #initilazition
      def __init__(self,config):
        super().__init__(config); # call the super with out config which is now pass..
        self.vocab_size = config.vocab_size; # embedding matrix row count
        # Build embedding of vocab size +1 x hidden size. +1 again because of padding.
        self.embedding = torch.nn.Embedding(num_embeddings=self.vocab_size+1,embedding_dim=config.hidden_size,padding_idx=0);
        torch.nn.init.uniform_(self.embedding.weight.data,-0.001,0.001); # initialization of the embedding values
        self.output = torch.nn.Linear(in_features=config.hidden_size,out_features=config.nlabels); # output layer is the size of the labels x hidden size.

      # forward
      def forward(self,input_ids,labels=None):
        embedded = self.embedding(input_ids); # sum up all the embeddings
        embedded_summed = torch.sum(embedded,dim=1); # sum up across word dimension
        projected = torch.tanh(embedded_summed); # non-linearity
        logits = self.output(projected); # apply the outer layer
      

        ## calculates the loss
        if labels is not None:
            # calculates the loss.
            loss = torch.nn.CrossEntropyLoss();
            return (loss(logits,labels),logits);
        else:
            # if no labels, just return the logits
            return (logits,);
  
# config
mlp_config = MLPConfig(vocab_size=len(vectorizer.vocabulary_),hidden_size=25,nlabels=6);

In [14]:
# training

# Set training arguments
trainer_args = transformers.TrainingArguments(
    "mlp_checkpoints", #save checkpoints here
    evaluation_strategy = "steps",
    logging_strategy = "steps",
    eval_steps = 500,
    logging_steps = 500,
    learning_rate = 1e-4, #learning rate of the gradient descent
    max_steps = 20000,
    load_best_model_at_end = True,
    per_device_train_batch_size = 128
)


# evaluating
accuracy = evaluate.load("accuracy");
def compute_accuracy(outputs_and_labels):
    outputs, labels = outputs_and_labels;
    predictions = np.argmax(outputs, axis=-1); #pick the index of the "winning" label
    return accuracy.compute(predictions=predictions, references=labels);

# actual training
mlp = MLP(mlp_config); # Make a the actual model  
early_stopping = transformers.EarlyStoppingCallback(5); # stop training if the eval loss is not getting better.

# params
trainer = transformers.Trainer(
    model = mlp,
    args = trainer_args,
    train_dataset = dset_tokenized["train"],
    eval_dataset = dset_tokenized["test"],
    compute_metrics = compute_accuracy,
    data_collator = collator,
    callbacks = [early_stopping]
)

# FINALLY!
trainer.train();


/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss,Validation Loss,Accuracy
500,1.640200,1.527431,0.457000
1000,1.446600,1.394457,0.557000
1500,1.262800,1.237796,0.603500
2000,1.060100,1.079695,0.679000
2500,0.870100,0.936762,0.754000
3000,0.707100,0.814198,0.805000
3500,0.574300,0.713081,0.836500
4000,0.469300,0.631903,0.857500
4500,0.387500,0.567124,0.863500
5000,0.323800,0.516251,0.869000


### 3.2 Hyperparameter optimization

In [28]:
# Used optuna for optimization

def objective(trial):
    # Define the search space for hyperparameters
    learning_rate = trial.suggest_float("learning_rate", 1e-10, 1e-1, log=True)
    batch_size = trial.suggest_categorical("batch_size", [8, 16, 64, 128, 256])
    epochs=trial.suggest_int('num_train_epochs', low = 2,high = 10),

    # params
    trainer_args = transformers.TrainingArguments(
        "mlp_checkpoints",
        evaluation_strategy = "steps",
        logging_strategy = "steps",
        eval_steps = 500,
        logging_steps = 500,
        learning_rate = learning_rate,
        max_steps = 30000,
        load_best_model_at_end = True,
        per_device_train_batch_size = batch_size,
        per_device_eval_batch_size = batch_size,
        num_train_epochs = epochs
    )

    # the model
    mlp = MLP(mlp_config)

    # train a model
    trainer = transformers.Trainer(
        model = mlp,
        args = trainer_args,
        train_dataset = dset_tokenized["train"],
        eval_dataset = dset_tokenized["validation"],
        compute_metrics = compute_accuracy,
        data_collator = collator,
        callbacks = [early_stopping]
    )

    trainer.train()
    eval_results = trainer.evaluate()
    return eval_results["eval_accuracy"] # return the best result.

study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=30)


[I 2023-04-17 14:01:23,657] A new study created in memory with name: no-name-1908305c-ec0c-4ddf-b696-bb956a232a8e
/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss,Validation Loss,Accuracy
500,1.784400,1.780946,0.089000
1000,1.783900,1.780847,0.089000
1500,1.784600,1.780750,0.089000
2000,1.783300,1.780658,0.089000
2500,1.783000,1.780564,0.089000
3000,1.784500,1.780474,0.089000
3500,1.783400,1.780386,0.089000
4000,1.783800,1.780298,0.089000
4500,1.783600,1.780212,0.089000
5000,1.783200,1.780129,0.089000


[I 2023-04-17 14:05:35,965] Trial 0 finished with value: 0.089 and parameters: {'learning_rate': 4.9906320835125987e-08, 'batch_size': 16, 'num_train_epochs': 8}. Best is trial 0 with value: 0.089.
/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss,Validation Loss,Accuracy
500,1.771700,1.774695,0.275000
1000,1.771000,1.773991,0.275000
1500,1.770200,1.773301,0.275000
2000,1.769500,1.772623,0.275000
2500,1.768800,1.771959,0.275000
3000,1.768100,1.771309,0.275000
3500,1.767400,1.770670,0.275000
4000,1.766700,1.770044,0.275000
4500,1.766000,1.769429,0.275000
5000,1.765400,1.768826,0.275000


[I 2023-04-17 14:09:28,331] Trial 1 finished with value: 0.275 and parameters: {'learning_rate': 2.934274792592448e-07, 'batch_size': 64, 'num_train_epochs': 10}. Best is trial 1 with value: 0.275.
/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


...

[I 2023-04-17 15:25:51,325] Trial 27 finished with value: 0.8985 and parameters: {'learning_rate': 0.000216232716038129, 'batch_size': 16, 'num_train_epochs': 4}. Best is trial 27 with value: 0.8985.
/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss,Validation Loss,Accuracy
500,1.692600,1.614051,0.413500
1000,1.571000,1.549140,0.460000
1500,1.503000,1.504785,0.491500
2000,1.469200,1.456803,0.541500
2500,1.387000,1.404245,0.555000
3000,1.340100,1.348633,0.571000
3500,1.254700,1.289192,0.585000
4000,1.190300,1.231338,0.604500
4500,1.104400,1.171133,0.617500
5000,1.049800,1.113245,0.650500


[I 2023-04-17 15:29:57,872] Trial 28 finished with value: 0.888 and parameters: {'learning_rate': 8.80238136501297e-05, 'batch_size': 16, 'num_train_epochs': 5}. Best is trial 27 with value: 0.8985.
/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss,Validation Loss,Accuracy
500,1.759800,1.751294,0.275000
1000,1.734100,1.724635,0.275000
1500,1.704700,1.697113,0.275000
2000,1.679300,1.672441,0.276000
2500,1.649800,1.649031,0.292500
3000,1.633600,1.629883,0.327000
3500,1.611200,1.613315,0.348000
4000,1.595600,1.599660,0.375000
4500,1.578500,1.588036,0.392000
5000,1.573400,1.578531,0.404000


[I 2023-04-17 15:34:00,334] Trial 29 finished with value: 0.5375 and parameters: {'learning_rate': 1.4428728455953327e-05, 'batch_size': 16, 'num_train_epochs': 4}. Best is trial 27 with value: 0.8985.


### 3.3. Evaluation on test set

In [145]:
# Your code to evaluate the final model on the test set here
eval_results = trainer.evaluate(dset_tokenized["validation"])

print(eval_results)

{'eval_loss': 0.33677756786346436, 'eval_accuracy': 0.8895, 'eval_runtime': 1.293, 'eval_samples_per_second': 1546.794, 'eval_steps_per_second': 193.349, 'epoch': 112.0}


Having fun :)

In [58]:
# creates a dataset with one example
def create_example(texts):
  labels = [];
  for item in texts:
    labels.append(0);
  data = {
      'text': texts,
      'label': labels,
  }
  return datasets.Dataset.from_dict(data);

In [151]:
# Testing random sentences that could be tweeted to see what they give as the result :)
texts = ['wow last night was so cool would recommend to everyone',
         'jesus the concert last night was so bad',
         'cant stand to stay at the school with these annoying people',
         'just got an F on the test',
         'just got an A on the test',
         'wow i never tought this day would come',
         'I just feel so useless every day',
         'wow finally feel like people really like me',
         'Just came from a date and it was so fun. He seems so nice and I can\'t wait to call him again',
         'i have been having a really good time with my friends and i feel like they really do care about me'
         'wait what just happened. omd I can\'t believe my eyes',
         'i was really suprised with the results',
         'tomorrow i have to go meet with my ex boyfriend. I just can\'t stand him anymore hes such an asshole',
         'he just suddenly fell infront of me I couldn\'t believe it',
         'I love him so much that I can\'t wait to marry him'];
labels = ['sadness', 'joy', 'love', 'anger', 'fear', 'suprise'];
example = create_example(texts); #creates a dict with just the example
example_tokenized = example.map(vectorize_item, num_proc=4); # tokenize
for i in range(len(texts)):
  print(texts[i])
  prediction = trainer.predict(example_tokenized).predictions[i]; # predicts the label
  largest = max(prediction); # label with largest value
  labelOfLargest = labels[list(prediction).index(largest)]; # name of label
  print(f"{labelOfLargest} with confidence of: {largest}\n");

Map (num_proc=4):   0%|          | 0/14 [00:00<?, ? examples/s]

wow last night was so cool would recommend to everyone


joy with confidence of: 4.247760772705078

jesus the concert last night was so bad


sadness with confidence of: 2.9032840728759766

cant stand to stay at the school with these annoying people


anger with confidence of: 0.6968567371368408

just got an F on the test


...

joy with confidence of: 2.719771385192871

i was really suprised with the results


joy with confidence of: 1.0176184177398682

tomorrow i have to go meet with my ex boyfriend. I just can't stand him anymore hes such an asshole


fear with confidence of: 1.5093954801559448

he just suddenly fell infront of me I couldn't believe it


sadness with confidence of: 0.9660069346427917

I love him so much that I can't wait to marry him


joy with confidence of: 1.9221845865249634



---

## 4. Results and summary

### 4.1 Corpus insights

(Briefly discuss what you learned about the corpus and its annotation)

The corpus includes Twitter messages in english and they have been annotated with six basic emotions which are anger, fear, joy, love, sadness, and surprise. 

By the paper the tweets have been selected with some hashtags and then annotated. The selected hastags can be seen from the paper.


### 4.2 Results

(Briefly summarize your results)

### 4.3 Relation to state of the art

(Compare your results to the state-of-the-art performance)

---

## 5. Bonus Task (optional)

### 5.1. Annotating out-of-domain documents

(Briefly describe the chosen out-of-domain documents)

(Briefly describe the process of annotation)

### 5.2 Conversion into dataset

In [19]:
# Your code to convert the annotations into a dataset here

### 5.3. Model evaluation on out-of-domain test set

In [20]:
# Your code to evaluate the model on the out-of-domain test set here

### 5.4 Bonus task results

(Present the results of the evaluation on the out-of-domain test set)

### 5.5. Annotated data

In [21]:
# Include your annotated out-of-domain data here